In [ ]:
%pip install -U insightface onnxruntime-gpu  numpy fastai pillow

In [ ]:
%pip install -U gdown moviepy ffmpeg insightface natsort hdbscan imutils chinese-whispers

In [ ]:
!sudo apt install ffmpeg

In [ ]:
import gdown
import cv2
import os
from moviepy.editor import VideoFileClip,AudioFileClip
import glob
import os.path as osp
import shutil
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

In [ ]:

# Define variables
drive_file_id = '1KOKqWBDMQzwgQwvvbQm06-DThmxys9Xk'  # replace 'FILE_ID' with your file's ID
video_output = 'my_video.mp4'
frame_folder = '/notebooks/frame_folder'
swapped_folder = '/notebooks/swapped'
character_folder = '/notebooks/character'


In [ ]:
shutil.rmtree(frame_folder)
shutil.rmtree(swapped_folder)

In [ ]:
assert insightface.__version__ >= '0.7'


# Create the frame folder if it doesn't exist
if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)
# Create the frame folder if it doesn't exist
if not os.path.exists(swapped_folder):
    os.makedirs(swapped_folder)
# Your face swapping script
app = FaceAnalysis(name='buffalo_l',providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))
swapper = insightface.model_zoo.get_model('/notebooks/inswapper_128.onnx')

In [ ]:


import numpy as np

def get_first_face(image):
    # If the image is a string (presumably a file path), read the image
    if isinstance(image, str):
        image = cv2.imread(image)

    # Check if the image is a valid numpy array
    if isinstance(image, np.ndarray):
        source_faces = app.get(image)
        source_faces = sorted(source_faces, key=lambda x: x.bbox[0])
        if len(source_faces) == 0:
            print(image)
            assert False

        return source_faces[0]
    print(image)
    assert False


In [ ]:
import requests

url = "http://example.com/path_to_your_file"  # replace with your file's URL
response = requests.get(url)

with open(video_output, 'wb') as f:  # replace with the path where you want to save the file
    f.write(response.content)

In [ ]:
# Download video from Google Drive
url = f'https://drive.google.com/uc?id={drive_file_id}'
gdown.download(url, video_output, quiet=False)

In [77]:
# Delete if swapped folder is not empty
if  os.path.exists(character_folder):
    shutil.rmtree(character_folder)

In [107]:
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import pickle
import datetime
# Create the frame folder if it doesn't exist
if not os.path.exists(character_folder):
    os.makedirs(character_folder)
if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)
# Initialize the FaceAnalysis application
""" app = FaceAnalysis(providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))  # Use GPU device 0 and input image size as (640, 640) """

# Open the video file
cap = cv2.VideoCapture(video_output)
# Calculate frame rate (frames per second)
fps = cap.get(cv2.CAP_PROP_FPS)
# Initialize a list to store embeddings and face data for each detected face
face_data = []
count = 0
# Loop through the video file frame by frame
while True:
    ret, frame = cap.read()
    if not ret:
        break
        # Calculate the time in minutes and seconds
    time_in_milliseconds = cap.get(cv2.CAP_PROP_POS_MSEC)
    frame_time = datetime.timedelta(milliseconds=time_in_milliseconds)
    path = os.path.join(frame_folder, f"{count}.jpg")
    cv2.imwrite(path, frame)  # save frame as JPEG file
    count += 1
    # Use the FaceAnalysis application to detect faces in the frame
    faces = app.get(frame)

    # For each detected face, extract the embedding, bounding box, and face image, and add them to the list
    for face in faces:
        if face.det_score < 0.1:
            continue
        face_embedding = face.embedding
       
        bbox = face.bbox.astype(int)
        cropped_face = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
    
        if cropped_face.size > 0:
            face_data.append({'label':'','frame':count,'time':frame_time,'face_embedding': face_embedding,'score':face.det_score,'normed_embedding': face.normed_embedding, 'bbox': bbox, 'image': cropped_face})
            

# Save the face_data to disk
with open('face_data.pkl', 'wb') as f:
    pickle.dump(face_data, f)


In [ ]:

def is_similar(face1, face2, threshold=0.5):
    # Extract normalized embeddings
    embedding1 = face1.normed_embedding
    embedding2 = face2.normed_embedding

    # Compute the dot product (cosine similarity, because embeddings are normalized)
    similarity = np.dot(embedding1, embedding2)

    # Return True if the similarity is above the threshold, False otherwise
    return similarity > threshold

In [110]:
# Delete if swapped folder is not empty
if  os.path.exists(character_folder):
    shutil.rmtree(character_folder)

In [ ]:
import os
import numpy as np
import cv2
import pickle
from imutils import build_montages
from sklearn.cluster import DBSCAN
from hdbscan import HDBSCAN
from sklearn.preprocessing import normalize


# Check if the directories exist
if not os.path.exists(character_folder):
    os.makedirs(character_folder)

# Load face data
with open('face_data.pkl', 'rb') as f:
    face_data = pickle.load(f)
times = np.array([d['time'].total_seconds() for d in face_data])
times = times / np.max(times)

""" # Extend your embeddings
for i, d in enumerate(face_data):
    d['normed_embedding'] = np.append(d['normed_embedding'], times[i]) """
embeddings = [d['normed_embedding'] for d in face_data]
norm_data = normalize(embeddings, norm='l2')

clustering = DBSCAN(eps=0.5, min_samples=6,metric="cosine")
clustering.fit(norm_data)

labels = np.unique(clustering.labels_)
print(labels)
for label in labels:
    # Skip the noise
    if label == -1:
        continue  

    directory_path = os.path.join(character_folder, str(label))
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    # Select samples associated with the current label
    idxs = np.where(clustering.labels_ == label)[0]
    idxs = np.random.choice(idxs, size=min(25, len(idxs)), replace=False)

    faces = []
    # loop over the sampled indexes
    for i in idxs:
        # Get the cropped face image from the 'image' key
        face = face_data[i]["image"]
        face_data[i]["label"] = str(label)
        time_elapsed= face_data[i]['time']
        # Calculate minutes and seconds
        minutes = time_elapsed.total_seconds() // 60
        seconds = time_elapsed.total_seconds() % 60

        # Format the time string as MM:SS
        frame_time = "{:02}:{:02}".format(int(minutes), int(seconds))
        # Force resize the face to 96x96 and then add it to the
        # faces montage list
        face = cv2.resize(face, (150, 150))
        cv2.putText(face,frame_time, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        faces.append(face)

        # Save face image
        image_path = os.path.join(directory_path, f"{i}.jpg")
        cv2.imwrite(image_path, face)
        
        # Save embedding
        face_embedding = face_data[i]['normed_embedding']
        embedding_path = os.path.join(directory_path, f"{i}.npy")
        np.save(embedding_path, face_embedding)
    # Create a montage using 96x96 "tiles" with 5 rows and 5 columns
    montage = build_montages(faces, (150, 150), (5, 5))[0]
    
    # Save the output montage
    title = "Face ID #{}".format(label)
    title = "Unknown Faces" if label == -1 else title
    cv2.imwrite(os.path.join(directory_path, title+'.jpg'), montage)
    new_face_data = new_face_data

# Save the face_data to disk
with open('face_data.pkl', 'wb') as f:
    pickle.dump(face_data, f)


[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52]


In [106]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from fastai.vision.all import *
from pathlib import Path
from PIL import Image as PilImage
import shutil
# Load face data
with open('face_data.pkl', 'rb') as f:
    face_data = pickle.load(f)

# Load data
path = Path(character_folder)

# Only consider directories where the name is numeric
folders = [dir for dir in path.iterdir() if dir.is_dir() and dir.name.isnumeric()]

labels = [folder.name for folder in folders]

# Create a dictionary to map old labels (folders) to new labels (characters)
label_dict = {}

# Define a generator to process one folder at a time
def process_folders():
    for label in labels:
        # Get a list of all image files in the cluster
        image_paths = list((path / label).glob('*.jpg'))
        if not image_paths:
            print(f"No images found in cluster: {label}")
            continue
        print(f"Current cluster: {label}")

        # Get the resolution of each image
        image_resolution = {}
        for image_path in image_paths:
            with PilImage.open(image_path) as img:
                width, height = img.size
                resolution = width * height  # resolution is defined as width * height
                image_resolution[image_path] = resolution

        # Sort images by resolution in descending order
        sorted_image_paths = sorted(image_resolution, key=image_resolution.get, reverse=True)

        # Display the largest image
        with open(sorted_image_paths[0], "rb") as file:
            image = file.read()
            img_widget = widgets.Image(value=image, format='png', width=1000, height=400)  # You might want to adjust the size
            display(img_widget)

        # Ask the user to input the name of the character
        name = widgets.Text(value=label, placeholder='Enter character name', description='Name:')
        button = widgets.Button(description='Confirm')
        display(name, button)

        def on_button_clicked(b):
            # Update the label dictionary
            label_dict[label] = name.value
            print(f'Character name confirmed: {name.value}')

            # Rename the folder
            old_name = Path(path / label)
            new_name = Path(path / name.value)

            # Update the label in the original face_data list
            for f in face_data:
                if 'label' in f and f['label'] == label:
                    f['label'] = name.value

            if new_name.exists():  # If the target directory exists
                # Copy all .jpg files in the current folder to the target folder
                for file in old_name.glob('*.jpg'):
                    if old_name != new_name:
                        shutil.copy(file, new_name / file.name)
                # Copy all .npy files (embeddings) in the current folder to the target folder
                for file in old_name.glob('*.npy'):
                    if old_name != new_name:
                        shutil.copy(file, new_name / file.name)
                # Delete the current folder
                shutil.rmtree(old_name)
            else:
                old_name.rename(new_name)
            # Save the face_data to disk
            with open('face_data.pkl', 'wb') as f:
                pickle.dump(face_data, f)
            # Proceed to the next cluster
            clear_output(wait=True)
            next(process)

        button.on_click(on_button_clicked)
  
        # Yield control and wait for the next call
        yield

# Start the process
process = process_folders()
next(process)  # Display the first cluster



Current cluster: 5


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Text(value='5', description='Name:', placeholder='Enter character name')

Button(description='Confirm', style=ButtonStyle())

In [76]:
from hmac import new
import pickle
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image as PilImage
import numpy as np
import io
import shutil
from pathlib import Path
from PIL import Image as PilImage
from ipywidgets import HBox, VBox
# Load face_data
with open('face_data.pkl', 'rb') as f:
    face_data = pickle.load(f)

cap = cv2.VideoCapture(video_output)

# Define path to the parent folder
path = Path(character_folder)  # replace with your folder path

# Define total frames for the slider
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define an output widget for the frame display
frame_output = widgets.Output()

# Define a generator to process one frame at a time
def process_frame(frame_number):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    ret, frame = cap.read()
    if not ret:
        return  # end of video
    # Convert frame (numpy array) to PIL image
    frame_pil = PilImage.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Resize the image
    base_width = 500
    w_percent = (base_width / float(frame_pil.size[0]))
    h_size = int((float(frame_pil.size[1]) * float(w_percent)))
    frame_pil = frame_pil.resize((base_width, h_size))

    # Convert PIL image to widget-compatible format
    img_byte_arr = io.BytesIO()
    frame_pil.save(img_byte_arr, format='PNG')
    frame_widget = widgets.Image(value=img_byte_arr.getvalue(), format='png')


    # Clear the output widget
    frame_output.clear_output()

    # Write to the output widget
    with frame_output:
        # Display the frame number and full frame
        print(f"Frame number: {frame_number}")
        """     display(frame_widget) """

        # Get face data corresponding to current frame
        faces = [p for p in face_data if p["frame"] == frame_number]
        # Create a list to store cropped face widgets
        face_widgets = []
        # Process each face in the frame
        for face in faces:
            # Display the time
            print(f"Time: {face['time']}")
            cropped_face = face['image']

            # Convert cropped_face (numpy array) to PIL image
            cropped_face_pil = PilImage.fromarray(cropped_face)

            # Convert PIL image to widget-compatible format
            img_byte_arr = io.BytesIO()
            cropped_face_pil.save(img_byte_arr, format='PNG')
            cropped_face_widget = widgets.Image(value=img_byte_arr.getvalue(), format='png')


            # Ask the user to input the name of the character
            name = widgets.Text(value=str(face['label']), placeholder='Enter character name', description='Name:')
            button = widgets.Button(description='Confirm')
            """     display(name, button) """

            def on_button_clicked(b):
                # Update the label dictionary
                label = name.value
                for f in face_data:
                    if f['frame'] == frame_number and np.array_equal(f['image'], face['image']):
                        f['label'] = name.value
                print(f'Character name confirmed: {name.value}')
                face['label'] = name.value
                # Rename the folder
                old_name = Path(path / label)
                new_name = Path(path / name.value)
                if new_name.exists():  # If the target directory exists
                    # Copy all .jpg files in the current folder to the target folder
                    for file in old_name.glob('*.jpg'):
                        if new_name != old_name:
                            shutil.copy(file, new_name / file.name)
                        # Copy all .npy files (embeddings) in the current folder to the target folder
                    for file in old_name.glob('*.npy'):
                        if new_name != old_name:
                            shutil.copy(file, new_name / file.name)
                        # Delete the current folder
                    shutil.rmtree(old_name)
                else:
                    if not os.path.exists(new_name):
                        os.mkdir(new_name)
                    old_name.rename(new_name)

                frame_output.clear_output(wait=True)
                process_frame(frame_number)
                with open('face_data.pkl', 'wb') as f:
                    pickle.dump(face_data, f)
                # Update the frame
                frame_output.clear_output(wait=True)

                # Get the faces in the current frame that have label '0'
                faces_with_label_0 = [f for f in face_data if f['frame'] == frame_number and f['label'] == '0']

                # If there are no more faces with label '0' in the current frame, proceed to the next frame
                if not faces_with_label_0:
                    next_frame_with_label_0 = [f for f in face_data if f['frame'] > frame_number and f['label'] == '0']
                    if next_frame_with_label_0:
                        next_frame_number = next_frame_with_label_0[0]['frame']
                        slider.value = next_frame_number  # Update the slider value
                    else:
                        print('No more frames with label 0.')
                else:
                    process_frame(frame_number)

            button.on_click(on_button_clicked)
            # Create a vertical box (VBox) for each face, including the image, text field, and button
            face_box = VBox([cropped_face_widget, name, button])

            # Add the face box to the list
            face_widgets.append(face_box)
        if len(faces) == 0:
            print("No face detected in this frame")
        # Display the cropped faces at the side of the main frame using HBox
        display(HBox([frame_widget] + face_widgets))
        
# At the start of your code, add the following line to get the first frame that has a label '0'
first_frame_with_label_0 = min([f['frame'] for f in face_data if f['label'] == '0'], default=0)
# Define a slider to navigate frames
slider = widgets.IntSlider(min=first_frame_with_label_0, max=total_frames, step=1, description='Frame:')

def on_slider_change(change):
    process_frame(change['new'])

slider.observe(on_slider_change, names='value')

# Display the slider and frame_output widget
display(slider, frame_output)


# ...

# Process the first frame with label '0'
process_frame(first_frame_with_label_0)

IntSlider(value=28, description='Frame:', max=899, min=28)

Output()

Character name confirmed: f1


Character name confirmed: f1


Character name confirmed: f1


...

Character name confirmed: m1


Character name confirmed: m1


Character name confirmed: m1


Character name confirmed: m1


Character name confirmed: m1


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import insightface
from insightface.app import FaceAnalysis
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

data = []
for label in os.listdir(character_folder):
    for filename in os.listdir(os.path.join(character_folder, label)):
        if filename.endswith('.npy'):  # check if the file is an image
            # read the image
            img_path = os.path.join(character_folder, label, filename)
            embedding = np.load(img_path)
            data.append({'embedding': embedding, 'label': label})
                

# create a dataframe
df = pd.DataFrame(data)

# Separate features and target
X = np.array(df['embedding'].to_list()) # Convert list of embeddings back to numpy array
y = df['label'].values

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Define the model
knn = KNeighborsClassifier(n_neighbors=4,metric='euclidean')

# Train the model
knn.fit(X_train, y_train)

# Validate the model
accuracy = knn.score(X_val, y_val)
print(f'Validation accuracy: {accuracy}')

In [ ]:
""" import pickle

# Open the pickle file and load the face data
with open('face_data.pkl', 'rb') as f:
    face_data = pickle.load(f)

# Now you can use the loaded face data. For example, you can extract all embeddings:
embeddings = np.array([data['face_embedding'] for data in face_data])


# Determine the optimal number of clusters, you might want to adjust this
n_clusters = 20

# Perform KMeans clustering
kmeans = KMeans(n_clusters=n_clusters).fit(embeddings)

# For each cluster center, find the closest face (in terms of Euclidean distance in the embedding space)
# and save it as a representative of the cluster (i.e., the character)
character_index= 0
previous = []
for i, center in enumerate(kmeans.cluster_centers_):
    distances = np.linalg.norm(embeddings - center, axis=1)
    closest_face_index = np.argmin(distances)
    bbox = face_data[closest_face_index]["bbox"]
    cropped_face = face_data[closest_face_index]["image"]
    found = False
    for index,j in enumerate(previous):
        similarity = np.dot(face_data[closest_face_index]["normed_embedding"], j)
        print(i, index,similarity)
        if similarity > 0.35:
            print("found same face")
            found = True
            break
    if not found:
        cv2.imwrite(f'{character_folder}/character_{character_index}.jpg',cropped_face)   
        previous.append(face_data[closest_face_index]["normed_embedding"])
        character_index += 1
from sklearn.neighbors import NearestNeighbors

# The input for NearestNeighbors will be the cluster centers.
# These act as the "labels" for each cluster.
nn_model = NearestNeighbors(n_neighbors=3, metric='cosine')
nn_model.fit(kmeans.cluster_centers_) """
def classify_face(embedding):
    predicted_label = knn.predict([embedding])
    predicted_label_str = le.inverse_transform(predicted_label)
    return predicted_label_str[0]


In [ ]:
if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)
# Extract frames from video
vidcap = cv2.VideoCapture(video_output)
success, image = vidcap.read()
count = 0
target_imgs = []  # a list of file paths to images of the target faces
while success:
    path = os.path.join(frame_folder, f"{count}.jpg")
    target_imgs.append(path)
    cv2.imwrite(path, image)  # save frame as JPEG file
    success, image = vidcap.read()
    count += 1

In [ ]:
# Delete if swapped folder is not empty
if  os.path.exists(swapped_folder):
    shutil.rmtree(swapped_folder)

In [ ]:
# Create the frame folder if it doesn't exist
if not os.path.exists(swapped_folder):
    os.makedirs(swapped_folder)

In [ ]:
import collections
import random
import natsort

swappings ={
   'm':get_first_face('/notebooks/1629085462605.jpeg'),
   'b':get_first_face('/notebooks/bcp.png'),
   'd':get_first_face('/notebooks/DSCAAZZEA.png'),
   'l2':get_first_face('/notebooks/2023-07-03 22.46.08.jpg'),
   'la':get_first_face('/notebooks/2023-07-03 22.47.10.jpg'),
   'jo':get_first_face('/notebooks/jocp.png'),
   'g':get_first_face('/notebooks/gacp.png'),
   'je':get_first_face('/notebooks/2023-07-03 22.47.24.jpg')
}
targets={
 'f': None
}

source_face = get_first_face('/notebooks/DSC06729.JPG')
image_files = glob.glob(os.path.join(frame_folder, '*.jpg'))  # get all jpg files in the frame folder
# Sort the file names using natural sorting
image_files = natsort.natsorted(image_files)
print(len(image_files))
# Initialize an empty dictionary to store the attribution of faces to people
attribution = {}
unused_keys = list(swappings.keys())
for key in list(targets.keys()):
    value = targets[key]
    if value is not None:
        unused_keys.remove(value)
# Define a deque for each face in the swappings dictionary with maxlen equal to the size of your smoothing window
smoothing_window_size = 5  # Or however large you want your smoothing window to be
embeddings_deques = collections.defaultdict(lambda: collections.deque(maxlen=smoothing_window_size))

# Now, in your loop over the image files, for each face you detect:
for img_file in image_files:
    img = cv2.imread(img_file)
    faces = app.get(img)
    faces = sorted(faces, key=lambda x: x.bbox[0])
    res = img.copy()
    for face in faces:
        if face.det_score < 0.1:
            continue
        found = False
        index = classify_face(face.normed_embedding)
        if index in targets:
            found = True
            value = targets[index]
            if value is not None:
                res = swapper.get(res, face, swappings[value], paste_back=True)
                break

        if not found:
            # Add the new embedding to the deque for that face
            embeddings_deques[index].append(face.normed_embedding)
            # Now use the mean of the embeddings in the deque to classify the face
            avg_embedding = np.mean(embeddings_deques[index], axis=0)
            index = classify_face(avg_embedding)

            if index in attribution:
                found = True
                res = swapper.get(res, face, swappings[attribution[index]], paste_back=True)
            else:
                print("index not found :" ,index)
        if not found:
            if len(unused_keys) == 0:  # If all faces have been used, reset the list
                unused_keys = list(swappings.keys())
                for key in list(targets.keys()):
                    if key in unused_keys:
                        unused_keys.remove(key)
            random_key = random.choice(unused_keys)
            unused_keys.remove(random_key)
            attribution[index] = random_key
            print("index ",index," attribute to ",random_key, " frame ", osp.basename(img_file))
            random_value = swappings[random_key]
            res = swapper.get(res, face, random_value, paste_back=True)
    cv2.imwrite(osp.join(swapped_folder, osp.basename(img_file)), res)



In [ ]:
import subprocess

output_video = "/notebooks/output.mp4"
video = cv2.VideoCapture(video_output)
fps = video.get(cv2.CAP_PROP_FPS)
video.release()

# Get the processed images
processed_images = glob.glob(os.path.join(swapped_folder, '*.jpg'))

# Sort the processed images (this may be necessary depending on how your files are named)
processed_images.sort()
print(len(processed_images))
# Initialize the video writer
height, width, _ = cv2.imread(processed_images[0]).shape
print(height, width)

# Define the command
command = f'ffmpeg -y -r {fps} -s {width}x{height} -i {swapped_folder}/%01d.jpg -vcodec libx264 -crf 25 -pix_fmt yuv420p {output_video}'

# Execute the command
subprocess.call(command, shell=True)

In [ ]:
import subprocess


temp_audio_file = '/notebooks/temp_audio.aac'

# Remove temporary audio file if it exists
if os.path.exists(temp_audio_file):
    os.remove(temp_audio_file)

# Remove output video with audio file if it exists
output_with_audio_file = '/notebooks/output_with_audio.mp4'
if os.path.exists(output_with_audio_file):
    os.remove(output_with_audio_file)

# Extract audio from original video and save it as a temporary audio file
audio_extraction_command = f'ffmpeg -y -i {video_output} -vn -acodec aac -strict -2 {temp_audio_file}'
subprocess.run(audio_extraction_command, shell=True)

# Combine swapped video with original audio
video_combination_command = f'ffmpeg -y -i {output_video} -i {temp_audio_file} -c:v copy -c:a copy -map 0:v:0 -map 1:a:0 {output_with_audio_file}'
subprocess.run(video_combination_command, shell=True)

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# path to the video file
video_path = output_with_audio_file

# specify the start and end times in seconds
# start time as (minutes, seconds)
start_time_min_sec = (0, 0)  # 2 minutes 30 seconds
start_time = start_time_min_sec[0]*60 + start_time_min_sec[1]

# end time as (minutes, seconds)
end_time_min_sec = (1, 40)  # 3 minutes 45 seconds
end_time = end_time_min_sec[0]*60 + end_time_min_sec[1]

# output file path
output_path = "/notebooks/split.mp4"

# extract subclip
ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=output_path)